# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [ ]:
%load_ext autoreload
%autoreload 2
import json
from datetime import datetime
from zoneinfo import ZoneInfo
from helpers.irp_integration import IRPClient
from helpers.sqlserver import execute_query_from_file, execute_script_file, display_result_sets

irp_client = IRPClient()

### Configurations

In [ ]:
# Metadata / Backend Config
server_name = "databridge-1"
connection_name = "DATABRIDGE"
peril = "USEQ"
date_value = "202511"
cycle_type = "Quarterly"

# SQL Scripts Workspace EDM
workspace_edm = "WORKSPACE_EDM"

# EDM Config
edm_name = f"RM_EDM_{peril}_{cycle_type}_{date_value}_Automated" # ensure unique within IRP
treaty_name = f"{peril}_Treaty"
edm_data_version = "22"

# MRI Import Config
mapping_file_name = "mapping.json"

# Portfolio Config
portfolio_name = f"{peril}_Automated" # ensure unique within the edm
geohaz_version = "22.0"

# Analysis Config
analysis_job_name = f"{peril} Analysis"
analysis_profile_name = f"DLM {peril} v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
event_rate_scheme_name = "RMS 2025 Stochastic Event Rates"
analysis_tag_name = f"{peril}_TAG"

# Grouping Config
group_name = f"RM_RDM_{date_value}_QEM_USAP"

# RDM Config
rdm_name = f"{peril}_Automated_RDM"

### Create EDM(s)

In [ ]:
job_ids = irp_client.edm.submit_create_edm_jobs(
    [
        {
            "edm_name": edm_name,
            "server_name": server_name
        }
    ]
)
print(json.dumps(irp_client.job.poll_risk_data_job_batch_to_completion(job_ids), indent=2))

### Create Portfolio(s)

In [ ]:
portfolio_ids = irp_client.portfolio.create_portfolios(
    [
        {
            "edm_name": edm_name,
            "portfolio_name": portfolio_name,
            "portfolio_number": portfolio_name,
            "description": f"{portfolio_name} created via IRP Notebook Framework"
        }
    ]
)
print(f"Created {len(portfolio_ids)} portfolios with IDs: {portfolio_ids}")

### MRI Import

In [ ]:
import_file_prefix = f"Modeling_{date_value}_Moodys"
import_file_value = "USEQ"

if import_file_value in ["CBHU", "CBEQ", "USFF", "USST", "USWF", "USHU_Full", "USHU_Leak"]:
    accounts_file_name = f"{import_file_prefix}_{import_file_value}_Account.csv"
    locations_file_name = f"{import_file_prefix}_{import_file_value}_Location.csv"
elif import_file_value == "USFL_Other":
    accounts_file_name = f"{import_file_prefix}_{cycle_type}_OtherFld_Account.csv"
    locations_file_name = f"{import_file_prefix}_{cycle_type}_OtherFld_Location.csv"
elif import_file_value == "USFL_Excess":
    accounts_file_name = f"{import_file_prefix}_{cycle_type}_ExcessFld_Account.csv"
    locations_file_name = f"{import_file_prefix}_{cycle_type}_ExcessFld_Location.csv"
elif import_file_value == "USFL_Commercial":
    accounts_file_name = f"{import_file_prefix}_{cycle_type}_ComFld_Account.csv"
    locations_file_name = f"{import_file_prefix}_{cycle_type}_ComFld_Location.csv"

import_response = irp_client.mri_import.import_from_files(
    edm_name=edm_name,
    portfolio_name=portfolio_name,
    accounts_file=accounts_file_name,
    locations_file=locations_file_name,
    mapping_file="mapping.json"
)

print(json.dumps(import_response, indent=2))

### Create Treaty

In [ ]:
treaty_ids = irp_client.treaty.create_treaties(
    [
        {
            "edm_name": edm_name,
            "treaty_name": treaty_name,
            "treaty_number": treaty_name,
            "treaty_type": "Working Excess",
            "per_risk_limit": 30000000.00,
            "occurrence_limit": 90000000.00,
            "attachment_point": 2000000.00,
            "inception_date": "2025-01-01T00:00:00.000Z",
            "expiration_date": "2030-01-01T00:00:00.000Z",
            "currency_name": "US Dollar",
            "attachment_basis": "Losses Occurring",
            "attachment_level": "Location",
            "pct_covered": 100.0,
            "pct_placed": 95.00,
            "pct_share": 100.00,
            "pct_retention": 100.00,
            "premium": 0.00,
            "num_reinstatements": 99,
            "pct_reinstatement_charge": 0.00,
            "aggregate_limit": 0.00,
            "aggregate_deductible": 0.00,
            "priority": 1
        }
    ]
)
print(f"Created {len(treaty_ids)} with IDs: {treaty_ids}")

### Upgrade EDM Version

In [ ]:
upgrade_job_ids = irp_client.edm.submit_upgrade_edm_data_version_jobs(
    [
        {
            "edm_name": edm_name,
            "edm_version": edm_data_version
        }
    ]
)
print(json.dumps(irp_client.edm.poll_data_version_upgrade_job_batch_to_completion(upgrade_job_ids), indent=2))

### Portfolio Mapping

In [ ]:
edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
if (len(edms) != 1):
    raise Exception(f"Found {len(edms)} EDMs with name {edm_name}; expected 1")
edm = edms[0]
exposure_id = edm['exposureId']
edm_full_name = edm['databaseName']
print(f'EDM Full Name: {edm_full_name}')

portfolios = irp_client.portfolio.search_portfolios(exposure_id, f"portfolioName=\"{portfolio_name}\"")
if (len(portfolios) != 1):
    raise Exception(f"Found {len(edms)} Portfolios with name {portfolio_name}; expected 1")
portfolio_id = portfolios[0]['portfolioId']
print(f'Portfolio ID: {portfolio_id}')
print()

sql_script_name = f"2b_Query_To_Create_Sub_Portfolios_{peril}_RMS_BackEnd.sql"
params = {
    "EDM_FULL_NAME": edm_full_name,
    "PORTFOLIO_ID": portfolio_id,
    "DATETIME_VALUE": datetime.now(ZoneInfo("America/New_York")).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
}

result = execute_script_file(
    file_path = f'portfolio_mapping/{sql_script_name}',
    params = params,
    connection = connection_name
)

print(result)

### GeoHaz

In [ ]:
geohaz_job_ids = irp_client.portfolio.submit_geohaz_jobs(
    [
        {
            "edm_name": edm_name,
            "portfolio_name": portfolio_name,
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        },
        {
            "edm_name": edm_name,
            "portfolio_name": 'USEQ_Manufactured',
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        },
        {
            "edm_name": edm_name,
            "portfolio_name": 'USEQ_Renters',
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        },
        {
            "edm_name": edm_name,
            "portfolio_name": 'USEQ_Condo',
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        },
        {
            "edm_name": edm_name,
            "portfolio_name": 'USEQ_CHFS',
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        },
        {
            "edm_name": edm_name,
            "portfolio_name": 'USEQ_Clay_21st',
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        },
        {
            "edm_name": edm_name,
            "portfolio_name": 'USEQ_Clay_Homes',
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        },
        {
            "edm_name": edm_name,
            "portfolio_name": 'USEQ_Other',
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        }
    ]
)
print(json.dumps(irp_client.portfolio.poll_geohaz_job_batch_to_completion(geohaz_job_ids), indent=2))

### Control Totals

In [ ]:
edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
if (len(edms) != 1):
    raise Exception(f"Found {len(edms)} EDMs with name {edm_name}; expected 1")
edm = edms[0]
edm_full_name = edm['databaseName']
print(f'Workspace EDM Full Name: {edm_full_name}')
print()

sql_script_name = '3d_RMS_EDM_CONTROL_Totals.sql'
params = {
    "WORKSPACE_EDM": edm_full_name,
    "CYCLE_TYPE": cycle_type,
    "DATE_VALUE": date_value
}

result = execute_query_from_file(
    file_path = f'control_totals/{sql_script_name}',
    params = params,
    connection = connection_name
)

display_result_sets(result)

### Execute Multiple Analyses

In [ ]:
analysis_job_ids = irp_client.analysis.submit_portfolio_analysis_jobs(
    [
        {
            "edm_name": edm_name,
            "job_name": "CBHU 2023 Rates",
            "portfolio_name": portfolio_name,
            "analysis_profile_name": analysis_profile_name,
            "output_profile_name": output_profile_name,
            "event_rate_scheme_name": "RMS 2023 Stochastic Event Rates",
            "treaty_names": [treaty_name],
            "tag_names": [analysis_tag_name]
        },
        {
            "edm_name": edm_name,
            "job_name": "CBHU 2025 Rates",
            "portfolio_name": portfolio_name,
            "analysis_profile_name": analysis_profile_name,
            "output_profile_name": output_profile_name,
            "event_rate_scheme_name": "RMS 2025 Stochastic Event Rates",
            "treaty_names": [treaty_name],
            "tag_names": [analysis_tag_name]
        }
    ]
)
analysis_batch_response = irp_client.analysis.poll_analysis_job_batch_to_completion(analysis_job_ids)
print(json.dumps(analysis_batch_response, indent=2))

### Group Analyses

In [ ]:
analysis_names = ["CBHU 2023 Rates", "CBHU 2025 Rates"]
grouping_job_ids = irp_client.analysis.submit_analysis_grouping_jobs(
    [
        {
            "group_name": group_name,
            "analysis_names": analysis_names
        }
    ]
)
grouping_response = irp_client.analysis.poll_analysis_grouping_job_batch_to_completion(grouping_job_ids)
print(json.dumps(grouping_response, indent=2))

### Export Analyses to RDM

In [ ]:
analysis_names = ["CBHU Group", "CBHU 2023 Rates", "CBHU 2025 Rates"]
rdm_export_response = irp_client.rdm.export_analyses_to_rdm(
    server_name=server_name,
    rdm_name=rdm_name,
    analysis_names=analysis_names
)
print(json.dumps(rdm_export_response, indent=2))

### Delete EDM

In [ ]:
print(json.dumps(irp_client.edm.delete_edm(edm_name), indent=2))